In [1]:
import os
import re
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
from metaphone import doublemetaphone
from pathlib import Path

#for Splink
from splink.exploratory import completeness_chart
from splink.blocking_analysis import count_comparisons_from_blocking_rule, n_largest_blocks,cumulative_comparisons_to_be_scored_from_blocking_rules_chart
from splink.exploratory import profile_columns
import splink.comparison_library as cl
from splink import block_on,Linker, SettingsCreator ,DuckDBAPI


In [2]:
def load_data(file_path:str) -> pd.DataFrame:
    """Load data from csv file
    
    Args:
    file_path: Path to CSV file
    
    Returns:
    pd.DataFrame: Dataframe loaded from csv file"""

    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"{file_path} not found.")
        
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        raise


In [3]:
df_a= load_data('file_a_with_nicknames.csv')
df_b = load_data('file_b_with_nicknames.csv')

In [4]:
def preview_data(df: pd.DataFrame):
    """Print basic information and preview of the dataframe."""
    display(df.head())
    print(df.info()) 

preview_data(df_a)
preview_data(df_b)

,first_name,last_name,street,city,state,zip,email,phone
0,Eric,Snyder,4073 Edwards Grove,South Kelly,AK,44727,rebecca88@shaw.info,NaN
1,Diana,Quinn,608 Ingram Field Suite 773,Lake Johnview,UT,33245,NaN,NaN
2,Alicia,Blair,3955 Mccoy Brooks,Kaylachester,CT,63338,samuel16@miller.info,1519515490
3,Michael,Lindsey,46712 Leblanc Ridges Suite 892,Garciamouth,NM,49877,whenderson@bell.com,NaN
4,Jared,Fischer,272 Mendez Path,Juanbury,AK,94246,pwhitaker@barber.com,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  1500 non-null   object
 1   last_name   1500 non-null   object
 2   street      1500 non-null   object
 3   city        1500 non-null   object
 4   state       1500 non-null   object
 5   zip         1500 non-null   int64 
 6   email       1013 non-null   object
 7   phone       1060 non-null   object
dtypes: int64(1), object(7)
memory usage: 93.9+ KB
None


,first_name,last_name,street,city,state,zip,email,phone
0,Amy,Terry,8702 Mitchell Freeway,Brianport,KS,42775,NaN,001-223-191-7251x8058
1,Steven,Martinez,158 Judith Centers,North Kevin,SC,81893,NaN,610.341.3123x74242
2,Becky,Herring,165 Alexis Plaza Suite 644,North David,MA,81944,NaN,280-333-0566x10336
3,Ashley,Ramos,68103 Roberts Flat,Ericton,AL,36979,rachel40@hotmail.com,561-618-1857x744
4,eniloraC,Martin,911 Gibson Walk Apt. 848,Alyssaside,NC,59341,kwalker@hotmail.com,8667332172


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  1000 non-null   object
 1   last_name   1000 non-null   object
 2   street      1000 non-null   object
 3   city        1000 non-null   object
 4   state       1000 non-null   object
 5   zip         1000 non-null   int64 
 6   email       702 non-null    object
 7   phone       697 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB
None


In [5]:
def clean_phone_number(phone):
    """
    Clean and standardize phone numbers to a consistent format.
    Returns a tuple of (cleaned_number, extension)
    
    Args:
        phone: The phone number string to clean
        
    Returns:
        pd.Series: Series with [cleaned_number, extension]
    """
    if pd.isnull(phone):
        return pd.Series([np.nan, np.nan])

    # Convert to string and remove whitespace
    phone_str = str(phone).lower().strip()

    # Regex to split extension (e.g., x123, ext.456)
    match = re.match(r'([^\dxext]*\d[\d\D]*?)(?:\s*(?:ext\.?|x)\s*(\d+))?$', phone_str)

    if match:
        main_part = re.sub(r'\D', '', match.group(1))  # remove non-digits
        extension = match.group(2) if match.group(2) else np.nan
        return pd.Series([main_part, extension])
    else:
        return pd.Series([re.sub(r'\D', '', phone_str), np.nan])


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize and clean input data
    
    Args:
        df: Input DataFrame
        
    Returns:
        pd.DataFrame: Preprocessed DataFrame
    """
    df = df.copy()

    # Clean text columns
    text_cols = ['first_name', 'last_name', 'street', 'city', 'state', 'email']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].str.strip().str.lower()

    # Format ZIP codes (with leading zeros)
    if 'zip' in df.columns:
        df['zip'] = df['zip'].astype(str).str.zfill(5)

    # Clean phone numbers
    if 'phone' in df.columns:
        df[['phone', 'phone_ext']] = df['phone'].apply(clean_phone_number)

    # Extract email domain
    if 'email' in df.columns:
        df['email'] = df['email'].str.lower()
        df['email_domain'] = df['email'].str.extract(r'@([\w\.-]+)')

    # To make entire DataFrame consistent
    df = df.where(pd.notnull(df), np.nan)

    # Create full name
    if 'first_name' in df.columns and 'last_name' in df.columns:
        df['full_name'] = df['first_name'].fillna('') + ' ' + df['last_name'].fillna('')
        df['full_name'] = df['full_name'].str.strip().str.lower()

    # Create phonetic encodings
    if 'first_name' in df.columns:
        df['first_name_metaphone'] = df['first_name'].apply(
            lambda x: doublemetaphone(x)[0] if pd.notnull(x) else np.nan
        )
    
    if 'last_name' in df.columns:
        df['last_name_metaphone'] = df['last_name'].apply(
            lambda x: doublemetaphone(x)[0] if pd.notnull(x) else np.nan
        )

    return df

In [6]:
preprocessed_data_a = preprocess_data(df_a)
preprocessed_data_b = preprocess_data(df_b)

In [7]:
preview_data(preprocessed_data_a)
preview_data(preprocessed_data_b)

,first_name,last_name,street,city,state,zip,email,phone,phone_ext,email_domain,full_name,first_name_metaphone,last_name_metaphone
0,eric,snyder,4073 edwards grove,south kelly,ak,44727,rebecca88@shaw.info,NaN,NaN,shaw.info,eric snyder,ARK,SNTR
1,diana,quinn,608 ingram field suite 773,lake johnview,ut,33245,NaN,NaN,NaN,NaN,diana quinn,TN,KN
2,alicia,blair,3955 mccoy brooks,kaylachester,ct,63338,samuel16@miller.info,1519515490,NaN,miller.info,alicia blair,ALS,PLR
3,michael,lindsey,46712 leblanc ridges suite 892,garciamouth,nm,49877,whenderson@bell.com,NaN,NaN,bell.com,michael lindsey,MKL,LNTS
4,jared,fischer,272 mendez path,juanbury,ak,94246,pwhitaker@barber.com,NaN,NaN,barber.com,jared fischer,JRT,FXR


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   first_name            1500 non-null   object
 1   last_name             1500 non-null   object
 2   street                1500 non-null   object
 3   city                  1500 non-null   object
 4   state                 1500 non-null   object
 5   zip                   1500 non-null   object
 6   email                 1013 non-null   object
 7   phone                 1060 non-null   object
 8   phone_ext             619 non-null    object
 9   email_domain          1013 non-null   object
 10  full_name             1500 non-null   object
 11  first_name_metaphone  1500 non-null   object
 12  last_name_metaphone   1500 non-null   object
dtypes: object(13)
memory usage: 152.5+ KB
None


,first_name,last_name,street,city,state,zip,email,phone,phone_ext,email_domain,full_name,first_name_metaphone,last_name_metaphone
0,amy,terry,8702 mitchell freeway,brianport,ks,42775,NaN,0012231917251,8058,NaN,amy terry,AM,TR
1,steven,martinez,158 judith centers,north kevin,sc,81893,NaN,6103413123,74242,NaN,steven martinez,STFN,MRTNS
2,becky,herring,165 alexis plaza suite 644,north david,ma,81944,NaN,2803330566,10336,NaN,becky herring,PK,HRNK
3,ashley,ramos,68103 roberts flat,ericton,al,36979,rachel40@hotmail.com,5616181857,744,hotmail.com,ashley ramos,AXL,RMS
4,enilorac,martin,911 gibson walk apt. 848,alyssaside,nc,59341,kwalker@hotmail.com,8667332172,NaN,hotmail.com,enilorac martin,ANLRK,MRTN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   first_name            1000 non-null   object
 1   last_name             1000 non-null   object
 2   street                1000 non-null   object
 3   city                  1000 non-null   object
 4   state                 1000 non-null   object
 5   zip                   1000 non-null   object
 6   email                 702 non-null    object
 7   phone                 697 non-null    object
 8   phone_ext             415 non-null    object
 9   email_domain          702 non-null    object
 10  full_name             1000 non-null   object
 11  first_name_metaphone  1000 non-null   object
 12  last_name_metaphone   1000 non-null   object
dtypes: object(13)
memory usage: 101.7+ KB
None


In [8]:
def combine_dataframes(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """Combine two preprocessed DataFrames, reset index, and add unique ID."""
    combined_df = pd.concat([df1, df2], ignore_index=True)
    combined_df.reset_index(drop=True)
    combined_df["unique_id"] = combined_df.index.astype(str)
    return combined_df

def identify_and_show_duplicates(df: pd.DataFrame, subset_cols: list) -> pd.DataFrame:
    """
    Create a 'duplicate_of' column showing the duplicate occurrences.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        subset_cols (list): List of column names to check for duplicates.
        
    Returns:
        pd.DataFrame: Updated DataFrame where 'duplicate_of' is 2 or more.
    """
    duplicates = df[df.duplicated(subset=subset_cols)]
    
    return duplicates


def analyze_completeness(df: pd.DataFrame, db_api: DuckDBAPI):
    """Generate completeness chart."""
    return completeness_chart(df, db_api=db_api)

def profile_data(df: pd.DataFrame, db_api: DuckDBAPI, top_n: int = 10, bottom_n: int = 5):
    """Profile selected columns using DuckDBAPI."""
    return profile_columns(df, db_api=db_api, top_n=top_n, bottom_n=bottom_n)


In [9]:
# Combine data
combined_df = combine_dataframes(preprocessed_data_a, preprocessed_data_b)

# Identify and show duplicates
duplicates = identify_and_show_duplicates(combined_df, subset_cols=["first_name", "last_name"])
print(f"Sample of duplicate records are: \n{duplicates} ")


Sample of duplicate records are: 
     first_name   last_name                         street            city  \
311       brian      taylor    97597 emma course suite 635    jamesborough   
328     timothy       perez              6424 tran freeway  north chadview   
484      nicole       smith    7801 zachary port suite 416     cynthiafurt   
662        jake    robinson     2756 ramirez row suite 031       robertton   
886        sean     ramirez          53770 callahan common       west lisa   
...         ...         ...                            ...             ...   
2294    matthew       smith     039 briggs course apt. 360       moraville   
2385    melissa       white  236 waters junctions apt. 883  anthonyborough   
2414    michael  fitzgerald             654 thomas circles     annettebury   
2467    tiffany      carter   27281 casey forges suite 849      jaredmouth   
2469      brian      taylor               70992 harper row  christopherton   

     state    zip            

In [10]:
# Analyze completeness of data
db_api = DuckDBAPI()
analyze_completeness(combined_df, db_api)

alt.LayerChart(...)

In [11]:
# Profile the data
profile_data(combined_df, db_api)

alt.VConcatChart(...)

In [12]:
# --- Blocking Rule Utilities ---

def create_blocking_rules() -> List[Tuple[str, str]]:
    """Create and return a list of named blocking rules."""
    return [
        ("first_name & last_name", block_on("first_name", "last_name")),
        ("zip", block_on("zip")),
        ("state", block_on("state")),
        ("city", block_on("city")),
        ("first_name_metaphone & last_name_metaphone", block_on("first_name_metaphone", "last_name_metaphone")),
        ("last_name_metaphone", block_on("last_name_metaphone")),
        ("first char of first_name", block_on("substr(first_name, 1,1)")),
        ("phone", block_on("phone")),
    ]

def count_blocking_rule_comparisons(
    df: pd.DataFrame,
    blocking_rules: List[Tuple[str, str]],
    db_api: DuckDBAPI,
    link_type: str = "dedupe_only"
) -> dict:
    """Count comparisons for each blocking rule."""
    results = {}
    for name, rule in blocking_rules:
        count = count_comparisons_from_blocking_rule(
            table_or_tables=df,
            blocking_rule=rule,
            link_type=link_type,
            db_api=db_api,
        )
        results[name] = count
    return results

def print_comparison_counts(counts: dict):
    """Print the comparison counts for each rule."""
    for name, count in counts.items():
        print(f"{name}: {count}")

def analyze_worst_case_blocks(
    df: pd.DataFrame,
    rules: List[str],
    db_api: DuckDBAPI,
    top_n: int = 5
):
    """Analyze largest blocks for given rule combinations."""
    for rule in rules:
        result = n_largest_blocks(
            table_or_tables=df,
            blocking_rule=block_on(*rule.split(",")),
            link_type="dedupe_only",
            db_api=db_api,
            n_largest=top_n
        )
        print(f"\nTop {top_n} blocks for rule: {rule}")
        display(result.as_pandas_dataframe())  # Only works in Jupyter/Notebook environments


In [13]:
blocking_rules = create_blocking_rules()
comparison_counts = count_blocking_rule_comparisons(combined_df, blocking_rules, db_api)
print_comparison_counts(comparison_counts)

rules_to_analyze = [
    "first_name,last_name",
    "first_name,last_name,first_name_metaphone,last_name_metaphone"
]
analyze_worst_case_blocks(combined_df, rules_to_analyze, db_api)

first_name & last_name: {'number_of_comparisons_generated_pre_filter_conditions': 3310, 'number_of_comparisons_to_be_scored_post_filter_conditions': 405, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."first_name" = r."first_name" AND l."last_name" = r."last_name"', 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}
zip: {'number_of_comparisons_generated_pre_filter_conditions': 3552, 'number_of_comparisons_to_be_scored_post_filter_conditions': 526, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."zip" = r."zip"', 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}
state: {'number_of_comparisons_generated_pre_filter_conditions': 125556, 'number_of_comparisons_to_be_scored_post_filter_conditions': 61528, 'filter_conditions_identified': '', 'equi_join_conditions_identified': 'l."state" = r."state"', 'link_type_join_condition': 'where l."unique_id" < r."unique_id"'}
city: {'number_of_comparisons_generated_p

,key_0,key_1,count_l,count_r,block_count
0,brian,johnson,4,4,16
1,timothy,perez,4,4,16
2,kelly,jackson,4,4,16
3,samantha,martinez,4,4,16
4,robert,sanchez,3,3,9



Top 5 blocks for rule: first_name,last_name,first_name_metaphone,last_name_metaphone


,key_0,key_1,key_2,key_3,count_l,count_r,block_count
0,brian,johnson,PRN,JNSN,4,4,16
1,timothy,perez,TM0,PRS,4,4,16
2,samantha,martinez,SMN0,MRTNS,4,4,16
3,kelly,jackson,KL,JKSN,4,4,16
4,james,smith,JMS,SM0,3,3,9


In [14]:
def create_settings():
    return SettingsCreator(
        link_type="dedupe_only",
        em_convergence=0.001,
        comparisons=[
            cl.JaroWinklerAtThresholds("first_name", [0.7, 0.85]),
            cl.JaroWinklerAtThresholds("last_name", [0.7, 0.85]),
            cl.ExactMatch("first_name_metaphone"),
            cl.ExactMatch("last_name_metaphone"),
            cl.EmailComparison("email"),
            cl.LevenshteinAtThresholds("phone", [0.7, 0.9]),
            cl.LevenshteinAtThresholds("street", [0.4, 0.7, 0.85]),
            cl.JaroWinklerAtThresholds("city", [0.4, 0.7, 0.85]),
            cl.ExactMatch("state"),
            cl.PostcodeComparison("zip"),
        ],
        blocking_rules_to_generate_predictions=[
            block_on("first_name", "last_name"),
            block_on("zip"),
            block_on("first_name_metaphone", "last_name_metaphone"),
            block_on("phone"),
            block_on("first_name", "zip"),
            block_on("last_name", "zip"),
            block_on("last_name_metaphone"),
        ],
        retain_intermediate_calculation_columns=True,
    )


In [15]:
def train_and_save_model(path: str,df: pd.DataFrame):
    settings = create_settings()
    linker = Linker(df, settings, db_api=DuckDBAPI())
    rules = [
        block_on("first_name", "last_name"),
        "levenshtein(r.last_name, l.last_name) <= 5 and levenshtein(r.first_name, l.first_name) <= 5",
    ]
    linker.training.estimate_probability_two_random_records_match(rules, recall=0.7)
    linker.training.estimate_u_using_random_sampling(max_pairs=1e9)
    for br in [block_on("first_name", "last_name"), block_on("zip"), block_on("city")]:
        linker.training.estimate_parameters_using_expectation_maximisation(br)

    linker.misc.save_model_to_json(path, overwrite=True)
    return linker


In [16]:
def visualize_model(linker):
    display(linker.visualisations.match_weights_chart())
    display(linker.visualisations.m_u_parameters_chart())
    display(linker.visualisations.parameter_estimate_comparisons_chart())

In [17]:
linker = train_and_save_model("./final_model.json",combined_df)

Probability two random records match is estimated to be  0.215.
This means that amongst all possible pairwise record comparisons, one in 4.65 are expected to match.  With 3,123,750 total possible comparisons, we expect a total of around 671,925.71 matching pairs
----- Estimating u probabilities using random sampling -----
u probability not trained for first_name - Jaro-Winkler distance of first_name >= 0.85 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for last_name - Jaro-Winkler distance of last_name >= 0.85 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for phone - Levenshtein distance of phone <= 0.7 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained for phone - Levenshtein distance of phone <= 0.9 (comparison vector va

In [18]:
display_of_model_visualizations = visualize_model(linker)

alt.VConcatChart(...)

alt.HConcatChart(...)

alt.Chart(...)

In [19]:
def generate_predictions(linker, prediction_path: str, cluster_path: str,threshold: float):
    df_predictions = linker.inference.predict()
    df_predictions_pd = df_predictions.as_pandas_dataframe() 
    df_predictions_pd = df_predictions_pd[df_predictions_pd["match_probability"] > threshold]
    df_predictions_pd.to_csv(prediction_path, index=False)

    clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
        df_predictions, threshold_match_probability=threshold)
    clusters_pd = clusters.as_pandas_dataframe()
    clusters_pd.to_csv(cluster_path, index=False)
    return df_predictions_pd, clusters_pd


In [20]:
df_preds, clusters = generate_predictions(linker, "splink_predictions.csv", "splink_clusters.csv",0.99)

Blocking time: 0.02 seconds
Predict time: 0.67 seconds

 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'first_name':
    u values not fully trained
Comparison: 'last_name':
    m values not fully trained
Comparison: 'last_name':
    u values not fully trained
Comparison: 'email':
    m values not fully trained
Comparison: 'phone':
    m values not fully trained
Comparison: 'phone':
    u values not fully trained
Comparison: 'street':
    m values not fully trained
Comparison: 'street':
    u values not fully trained
Comparison: 'city':
    m values not fully trained
Comparison: 'city':
    u values not fully trained
Comparison: 'zip':
    m values not fully trained
Comparison: 'zip':
    u values not fully trained
Compl

In [21]:
def get_deduped_id_mapping(df: pd.DataFrame) -> Dict[int, int]:
    """Create a mapping of deduplicated IDs, mapping unique_id_r to the smallest unique_id_l."""
    deduped_ids = df.groupby('unique_id_r')['unique_id_l'].first().reset_index()
    id_mapping = pd.Series(deduped_ids['unique_id_l'].values, index=deduped_ids['unique_id_r']).to_dict()
    int_id_mapping = {int(k): int(v) for k, v in id_mapping.items()}
    return int_id_mapping


def deduplicate_by_mapped_ids(df: pd.DataFrame,column_name:str,id_mapping,output_path:str) -> None:
    """Convert the 'unique_id' column to integer type if necessary."""
    if df[column_name].dtype != 'int64':
        df[column_name] = df[column_name].astype(int)
    df.loc[:, column_name] = df[column_name].replace(id_mapping)
    df = df.drop_duplicates(subset=column_name, keep='last')
    df.to_csv(output_path,index=False)



In [22]:
int_mapping = get_deduped_id_mapping(df_preds)
deduplicate_by_mapped_ids(combined_df,"unique_id",int_mapping,"merged_consumers.csv")